# **Evaluating Groundedness using LLMs**

### Overview
In this demonstration, you will evaluate the groundedness of the context for a RAG architecture pattern. RAG (Retrieval Augmented Generation) is a common method used to improve LLM results and reduce hallucnations by 'grounding' the LLM resposne using information retrieval. There are generally to types og "Groundedness" metrics:  
1. **Question Groundedness** - Measures the quality of the search index. "How good is the data being retrieved for a given question?"
2. **Answer Groundedness** - Measures the quality of LLM response. "How good is my LLM at using the data being retrieved?"

After utilizing Azure PromptFlow to generate and evaluate RAG chat responses, this notebook will take a deeper look at the results.

 **_Go Deeper_**  
- [Are Large Language Model-based Evaluators the Solution to Scaling Up Multilingual Evaluation?](https://ar5iv.labs.arxiv.org/html/2309.07462)
- [GptEval: NLG Evaluation using Gpt-4 with Better Human Alignment](https://ar5iv.labs.arxiv.org/html/2303.16634)
  
**_Prerequisites_**  
  
Ensure that your environment is setup by completing the steps outlines in [0_setup.ipynb](./0_setup.ipynb)

## 1. Upload Sample Input Data

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

import os

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import time

#TODO create sample data
local_path = "XXX"
# set the version number of the data asset to the current UTC time
v1 = time.strftime("%Y.%m.%d.%H%M%S", time.gmtime())


my_data = Data(
    name="simple-chat-sample-inputs",
    version=v1,
    description="Sample inputs for simple chat flow",
    path=local_path,
    type=AssetTypes.URI_FILE,
)

# create data asset
ml_client.data.create_or_update(my_data)

print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")

## 2. Run Simple Chat & GPT Evaluation PromptFlow Jobs
In this section you will run a [sample RAG chat](../src/promptflow/sample_chat_flows/simplrag_chat) against a small sample dataset  

Then, as part of the same job, you will evaluate the GPT metrics above using an [evaluation PromptFlow](../src/promptflow/evaluation_flows/rag_gpt_eval/)

Both the simple chat and the evaluation utilize the AOAI connection established during setup and cooresponding GPT4 deployment

##### **IMPORTANT**: _Please take a moment to analyze in depth the Simple Chat, Evaluation Flow, and the sample dataset linked above_

In [ ]:
from promptflow import PFClient

# PFClient can help manage your runs and connections.
pf = PFClient()

# Define Flows and Data
simple_chat_flow = "../src/promptflow/sample_chat_flows/rag_chat" # set the flow directory
eval_flow = "../src/promptflow/evaluation_flows/rag_gpt_eval" # set flow directory
# TODO data = "../data/inputs/simple_chat_sample_inputs.csv" # set the data file

# Run chat flow to generate chat results
chat_run = pf.run(
    flow=simple_chat_flow,
    data=data,
    stream=False,
    column_mapping={  # map the url field from the data to the url input of the flow
      "input": "${data.input}",
    }
)

# Run evaluation flow to evaluate chat results
eval_run = pf.run(
    flow=eval_flow,
    data=data,
    run=chat_run,
    stream=False,
    column_mapping={  # map the url field from the data to the url input of the flow
      "question": "${data.input}",
      "response": "${run.outputs.output}",
    }
)


## 3.  View Results  
To view outputs in detail analyze the [output data](../data/outputs/rag_eval_results.json) directly